In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn
import pandas as pd
import os
import sys
import time
import tensorflow as tf

from tensorflow import keras

print(tf.__version__)
print(sys.version_info)
for module in mpl, np, pd, sklearn, tf, keras:
    print(module.__name__, module.__version__)

2.6.4
sys.version_info(major=3, minor=7, micro=12, releaselevel='final', serial=0)
matplotlib 3.5.2
numpy 1.21.6
pandas 1.3.5
sklearn 1.0.2
tensorflow 2.6.4
keras.api._v2.keras 2.6.0


In [2]:
input_filepath = '../input/shakespeare/shakespeare.txt'
text = open(input_filepath, 'r').read()
print(len(text))
print(text[:100])

1115394
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [3]:
vocab = sorted(set(text))
print(len(vocab))
print(vocab)

65
['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [4]:
char2index = {char: index for index, char in enumerate(vocab)}
print(char2index)

{'\n': 0, ' ': 1, '!': 2, '$': 3, '&': 4, "'": 5, ',': 6, '-': 7, '.': 8, '3': 9, ':': 10, ';': 11, '?': 12, 'A': 13, 'B': 14, 'C': 15, 'D': 16, 'E': 17, 'F': 18, 'G': 19, 'H': 20, 'I': 21, 'J': 22, 'K': 23, 'L': 24, 'M': 25, 'N': 26, 'O': 27, 'P': 28, 'Q': 29, 'R': 30, 'S': 31, 'T': 32, 'U': 33, 'V': 34, 'W': 35, 'X': 36, 'Y': 37, 'Z': 38, 'a': 39, 'b': 40, 'c': 41, 'd': 42, 'e': 43, 'f': 44, 'g': 45, 'h': 46, 'i': 47, 'j': 48, 'k': 49, 'l': 50, 'm': 51, 'n': 52, 'o': 53, 'p': 54, 'q': 55, 'r': 56, 's': 57, 't': 58, 'u': 59, 'v': 60, 'w': 61, 'x': 62, 'y': 63, 'z': 64}


In [5]:
index2char = np.array(vocab)
print(index2char)

['\n' ' ' '!' '$' '&' "'" ',' '-' '.' '3' ':' ';' '?' 'A' 'B' 'C' 'D' 'E'
 'F' 'G' 'H' 'I' 'J' 'K' 'L' 'M' 'N' 'O' 'P' 'Q' 'R' 'S' 'T' 'U' 'V' 'W'
 'X' 'Y' 'Z' 'a' 'b' 'c' 'd' 'e' 'f' 'g' 'h' 'i' 'j' 'k' 'l' 'm' 'n' 'o'
 'p' 'q' 'r' 's' 't' 'u' 'v' 'w' 'x' 'y' 'z']


In [6]:
text_as_int = np.array([char2index[c] for c in text])
print(text_as_int.shape)
print(len(text_as_int))
print(text_as_int[:10])
print(text[:10])

(1115394,)
1115394
[18 47 56 57 58  1 15 47 58 47]
First Citi


In [7]:
def split_input_target(id_text):
    """abcde -> abcd,bcde  输入是abcd，输出是bcde"""
    return id_text[0:-1], id_text[1:]

In [8]:
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
sequence_length = 100
# 输入sequence_length+1各字符 返回sequence_length个字符
# batch将字符转换为句子序列  drop_remainder丢掉最后不足一批的余数
sequence_dataset = char_dataset.batch(sequence_length + 1, drop_remainder=True)

for char_id in char_dataset.take(2):
    print(char_id, index2char[char_id.numpy()])
for sequence_id in sequence_dataset.take(2):
    print(sequence_id)
    print(repr(''.join(index2char[sequence_id.numpy()])))

tf.Tensor(18, shape=(), dtype=int64) F
tf.Tensor(47, shape=(), dtype=int64) i
tf.Tensor(
[18 47 56 57 58  1 15 47 58 47 64 43 52 10  0 14 43 44 53 56 43  1 61 43
  1 54 56 53 41 43 43 42  1 39 52 63  1 44 59 56 58 46 43 56  6  1 46 43
 39 56  1 51 43  1 57 54 43 39 49  8  0  0 13 50 50 10  0 31 54 43 39 49
  6  1 57 54 43 39 49  8  0  0 18 47 56 57 58  1 15 47 58 47 64 43 52 10
  0 37 53 59  1], shape=(101,), dtype=int64)
'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
tf.Tensor(
[39 56 43  1 39 50 50  1 56 43 57 53 50 60 43 42  1 56 39 58 46 43 56  1
 58 53  1 42 47 43  1 58 46 39 52  1 58 53  1 44 39 51 47 57 46 12  0  0
 13 50 50 10  0 30 43 57 53 50 60 43 42  8  1 56 43 57 53 50 60 43 42  8
  0  0 18 47 56 57 58  1 15 47 58 47 64 43 52 10  0 18 47 56 57 58  6  1
 63 53 59  1 49], shape=(101,), dtype=int64)
'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'


2022-08-03 09:36:20.387711: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-03 09:36:20.541245: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-03 09:36:20.542193: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-03 09:36:20.549293: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

In [9]:
# 做映射得到输入和输出
sequence_dataset = sequence_dataset.map(split_input_target)
for item_input, item_output in sequence_dataset.take(2):
    print(item_input)
    print(item_output)
print(sequence_dataset)

2022-08-03 09:36:23.537076: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


tf.Tensor(
[18 47 56 57 58  1 15 47 58 47 64 43 52 10  0 14 43 44 53 56 43  1 61 43
  1 54 56 53 41 43 43 42  1 39 52 63  1 44 59 56 58 46 43 56  6  1 46 43
 39 56  1 51 43  1 57 54 43 39 49  8  0  0 13 50 50 10  0 31 54 43 39 49
  6  1 57 54 43 39 49  8  0  0 18 47 56 57 58  1 15 47 58 47 64 43 52 10
  0 37 53 59], shape=(100,), dtype=int64)
tf.Tensor(
[47 56 57 58  1 15 47 58 47 64 43 52 10  0 14 43 44 53 56 43  1 61 43  1
 54 56 53 41 43 43 42  1 39 52 63  1 44 59 56 58 46 43 56  6  1 46 43 39
 56  1 51 43  1 57 54 43 39 49  8  0  0 13 50 50 10  0 31 54 43 39 49  6
  1 57 54 43 39 49  8  0  0 18 47 56 57 58  1 15 47 58 47 64 43 52 10  0
 37 53 59  1], shape=(100,), dtype=int64)
tf.Tensor(
[39 56 43  1 39 50 50  1 56 43 57 53 50 60 43 42  1 56 39 58 46 43 56  1
 58 53  1 42 47 43  1 58 46 39 52  1 58 53  1 44 39 51 47 57 46 12  0  0
 13 50 50 10  0 30 43 57 53 50 60 43 42  8  1 56 43 57 53 50 60 43 42  8
  0  0 18 47 56 57 58  1 15 47 58 47 64 43 52 10  0 18 47 56 57 58  6  1
 63 53 

In [10]:
batch_size = 64
buffer_size = 10000
# 选buffer_size个数据进行shuffle可以避免内存不足，从缓冲区中随机读出一个后，从原数据集的顺序元素补入缓冲区
sequence_dataset = sequence_dataset.shuffle(buffer_size).batch(batch_size, drop_remainder=True)
print(sequence_dataset)

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>


In [11]:
print(len(sequence_dataset))
print(len(text_as_int)//(sequence_length + 1)//batch_size)

172
172


In [12]:
vocab_size = len(vocab)
embedding_dim = 256 # 资料较小 增大维度
rnn_units = 1024


def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = keras.models.Sequential([
        keras.layers.Embedding(vocab_size, embedding_dim, 
                               batch_input_shape=[batch_size, None]),# None表示不定长序列
        keras.layers.SimpleRNN(units=rnn_units,
                               stateful=True,  # 是否把上一批最后返回的状态添加到下一批作为输入
                               recurrent_initializer='glorot_uniform',
                               return_sequences=True),  # 返回所有输出
        keras.layers.Dense(vocab_size),
    ])
    return model


model = build_model(vocab_size=vocab_size, embedding_dim=embedding_dim,
                    rnn_units=rnn_units, batch_size=batch_size)
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           16640     
_________________________________________________________________
simple_rnn (SimpleRNN)       (64, None, 1024)          1311744   
_________________________________________________________________
dense (Dense)                (64, None, 65)            66625     
Total params: 1,395,009
Trainable params: 1,395,009
Non-trainable params: 0
_________________________________________________________________
None


In [13]:
print(model.variables)

[<tf.Variable 'embedding/embeddings:0' shape=(65, 256) dtype=float32, numpy=
array([[ 0.0315897 ,  0.0191695 , -0.02248891, ...,  0.00252968,
        -0.0051024 ,  0.03776758],
       [-0.03473364,  0.00246806, -0.04956435, ..., -0.0260577 ,
        -0.04053782, -0.04774687],
       [ 0.02023702,  0.02073567, -0.01848514, ...,  0.01088298,
         0.02291042,  0.03920123],
       ...,
       [ 0.00585954, -0.04024822, -0.02104069, ...,  0.02204395,
        -0.035839  ,  0.02105465],
       [ 0.03400784, -0.00209029,  0.01029694, ..., -0.00064523,
         0.03632439,  0.00841641],
       [-0.0074083 ,  0.02868542, -0.00435374, ...,  0.03057984,
         0.01815863,  0.03489191]], dtype=float32)>, <tf.Variable 'simple_rnn/simple_rnn_cell/kernel:0' shape=(256, 1024) dtype=float32, numpy=
array([[-0.03398598,  0.02765952,  0.05117539, ..., -0.05452574,
         0.04890401, -0.02409996],
       [-0.06255478, -0.00223817, -0.05884085, ..., -0.01992139,
        -0.06477816,  0.00636961],
  

In [14]:
for input_example_batch, target_example_batch in sequence_dataset.take(1):
    # 把model当函数来用，实际是调用类的call方法
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions)

tf.Tensor(
[[[-0.0277677  -0.00307428 -0.01136489 ... -0.02350267  0.01461161
    0.0019732 ]
  [ 0.01010692  0.04443358  0.05428067 ...  0.02439065  0.02949676
    0.02530561]
  [-0.03607504 -0.01973029  0.07881797 ...  0.01662827 -0.01038327
    0.00734811]
  ...
  [ 0.11734699  0.39395094  0.0251657  ...  0.10838296  0.1353434
    0.00128112]
  [-0.08809775  0.06264967 -0.03204473 ...  0.03523269 -0.01832261
   -0.01624823]
  [-0.18793745 -0.2094123  -0.11055788 ...  0.06602431  0.15891182
    0.07269562]]

 [[ 0.01691867  0.02228602  0.02628811 ... -0.02378176 -0.01417266
    0.03836555]
  [ 0.02831957 -0.00686338  0.02477144 ... -0.05904023  0.01052604
    0.01227409]
  [-0.03546701 -0.02310277 -0.02865003 ...  0.00533582  0.01301381
   -0.00470249]
  ...
  [-0.03440933  0.36867595 -0.29875845 ... -0.09362227  0.02077944
   -0.01168906]
  [-0.10252287  0.08922808  0.11379665 ...  0.20371579  0.1979198
   -0.23882915]
  [ 0.26593766  0.16142857  0.02293691 ... -0.06974638 -0.012070

In [15]:
print(example_batch_predictions[0][0])

tf.Tensor(
[-0.0277677  -0.00307428 -0.01136489 -0.02592429 -0.03541304  0.05327918
  0.03731832  0.03630592 -0.02428248 -0.02034934 -0.00110926 -0.02361733
 -0.00488487  0.0073754  -0.00180079  0.03579012  0.02962378  0.02126725
  0.02433565 -0.02989892 -0.00509168  0.04281706  0.03638208  0.00128084
  0.00827455  0.00774063 -0.02448355  0.02234148  0.05311988  0.04340787
 -0.02852925 -0.0302019  -0.04263079  0.03580119 -0.00705556  0.02946792
 -0.00340241  0.00376599  0.03039584 -0.00544588  0.00809595  0.01556165
 -0.05214859 -0.05935791 -0.01086076 -0.01550933 -0.00858685 -0.04007464
 -0.04232285 -0.02414624  0.0251887  -0.02858887 -0.0092003   0.01913104
  0.02768249  0.00738442 -0.00983901  0.01824012 -0.04077822 -0.04698012
  0.05142738  0.00080016 -0.02350267  0.01461161  0.0019732 ], shape=(65,), dtype=float32)


In [16]:
print(example_batch_predictions[0])

tf.Tensor(
[[-0.0277677  -0.00307428 -0.01136489 ... -0.02350267  0.01461161
   0.0019732 ]
 [ 0.01010692  0.04443358  0.05428067 ...  0.02439065  0.02949676
   0.02530561]
 [-0.03607504 -0.01973029  0.07881797 ...  0.01662827 -0.01038327
   0.00734811]
 ...
 [ 0.11734699  0.39395094  0.0251657  ...  0.10838296  0.1353434
   0.00128112]
 [-0.08809775  0.06264967 -0.03204473 ...  0.03523269 -0.01832261
  -0.01624823]
 [-0.18793745 -0.2094123  -0.11055788 ...  0.06602431  0.15891182
   0.07269562]], shape=(100, 65), dtype=float32)


In [17]:
# logits是网络最终的全连接层的输出 还未经过sigmoid或者softmax的概率化  num_samples抽样次数
# tf.random.categorical从分类分布中随机抽取num_samples个样本返回下标,随机为了每次生成文本有差异
sample_indices = tf.random.categorical(
    logits=example_batch_predictions[0], num_samples=1, seed=1)
print(sample_indices.shape) # (100, 1)
print(sample_indices)  
print('-' * 50)
sample_indices = tf.squeeze(sample_indices, axis=1)
print(sample_indices)  # (100, )

(100, 1)
tf.Tensor(
[[31]
 [63]
 [64]
 [23]
 [51]
 [15]
 [54]
 [ 1]
 [48]
 [64]
 [17]
 [32]
 [59]
 [ 5]
 [40]
 [44]
 [ 3]
 [46]
 [19]
 [53]
 [56]
 [18]
 [ 6]
 [25]
 [ 8]
 [ 5]
 [41]
 [19]
 [15]
 [38]
 [63]
 [ 7]
 [18]
 [38]
 [56]
 [64]
 [44]
 [45]
 [31]
 [64]
 [54]
 [41]
 [45]
 [34]
 [61]
 [41]
 [40]
 [45]
 [36]
 [40]
 [28]
 [62]
 [56]
 [18]
 [35]
 [14]
 [44]
 [58]
 [58]
 [53]
 [53]
 [14]
 [27]
 [41]
 [28]
 [10]
 [36]
 [56]
 [46]
 [24]
 [11]
 [28]
 [ 0]
 [62]
 [24]
 [ 6]
 [ 6]
 [62]
 [18]
 [32]
 [44]
 [56]
 [13]
 [13]
 [48]
 [34]
 [23]
 [ 1]
 [ 7]
 [50]
 [55]
 [60]
 [63]
 [12]
 [ 4]
 [51]
 [61]
 [29]
 [21]
 [39]], shape=(100, 1), dtype=int64)
--------------------------------------------------
tf.Tensor(
[31 63 64 23 51 15 54  1 48 64 17 32 59  5 40 44  3 46 19 53 56 18  6 25
  8  5 41 19 15 38 63  7 18 38 56 64 44 45 31 64 54 41 45 34 61 41 40 45
 36 40 28 62 56 18 35 14 44 58 58 53 53 14 27 41 28 10 36 56 46 24 11 28
  0 62 24  6  6 62 18 32 44 56 13 13 48 34 23  1  7 50 55 60 63 12  

In [18]:
for i in tf.range(5):
    # 虽然是随机的，但是还是偏向于概率较大的值
    samples = tf.random.categorical([[4.0, 2.0, 2.0, 2.0, 1.0]], 3)
    tf.print(samples)

[[0 0 3]]
[[0 0 0]]
[[3 0 0]]
[[1 0 0]]
[[1 0 0]]


In [19]:
print('Input:', repr(''.join(index2char[input_example_batch[0]])))
print('-' * 50)
print('Output:', repr(''.join(index2char[target_example_batch[0]])))
print('-' * 50)
print('Predictions:', repr(''.join(index2char[sample_indices])))

Input: "ward on the journey you shall go.\n\nBARNARDINE:\nI swear I will not die to-day for any man's\npersuasio"
--------------------------------------------------
Output: "ard on the journey you shall go.\n\nBARNARDINE:\nI swear I will not die to-day for any man's\npersuasion"
--------------------------------------------------
Predictions: "SyzKmCp jzETu'bf$hGorF,M.'cGCZy-FZrzfgSzpcgVwcbgXbPxrFWBfttooBOcP:XrhL;P\nxL,,xFTfrAAjVK -lqvy?&mwQIa"


In [20]:
# from_logits是否预期为对数张量。默认情况False下，输出的logits需要经过激活函数的处理再传入接口中
# from_logits = False 表示输入进来的y_pred已符合某种分布(即输出层是带softmax激活函数的), 接口只会帮你再进行概率归一化
# from_logits = True 表示是原始数据，接口会帮你做softmax后再进行计算
def loss(labels, logits):
    return keras.losses.sparse_categorical_crossentropy(
        labels, logits, from_logits=True)


model.compile(optimizer='adam', loss=loss)
example_loss = loss(target_example_batch, example_batch_predictions)
print(example_loss.shape)
print(example_loss.numpy().mean())

(64, 100)
4.174875


In [21]:
# 保存模型
output_dir = './text_generation_checkpoints'
if not os.path.exists(output_dir):
    os.mkdir(output_dir)
checkpoint_prefix = os.path.join(output_dir, 'ckpt_{epoch}')
checkpoint_callback = keras.callbacks.ModelCheckpoint(filepath=checkpoint_prefix,
                                                      # 只保存权重的值
                                                      save_weights_only=True)
epochs = 100
history = model.fit(sequence_dataset, epochs = epochs,
                    callbacks = [checkpoint_callback])

Epoch 1/100
172/172 [==============================] - 18s 89ms/step - loss: 2.6431
Epoch 2/100
172/172 [==============================] - 17s 87ms/step - loss: 2.2587
Epoch 3/100
172/172 [==============================] - 17s 85ms/step - loss: 1.9799
Epoch 4/100
172/172 [==============================] - 18s 90ms/step - loss: 1.8402
Epoch 5/100
172/172 [==============================] - 17s 89ms/step - loss: 1.7368
Epoch 6/100
172/172 [==============================] - 18s 95ms/step - loss: 1.6613
Epoch 7/100
172/172 [==============================] - 17s 86ms/step - loss: 1.6035
Epoch 8/100
172/172 [==============================] - 18s 86ms/step - loss: 1.5577
Epoch 9/100
172/172 [==============================] - 17s 88ms/step - loss: 1.5218
Epoch 10/100
172/172 [==============================] - 17s 85ms/step - loss: 1.4937
Epoch 11/100
172/172 [==============================] - 18s 90ms/step - loss: 1.4697
Epoch 12/100
172/172 [==============================] - 18s 92ms/step - lo

In [22]:
output_dir = "./text_generation_checkpoints"
print(tf.train.latest_checkpoint(output_dir))

./text_generation_checkpoints/ckpt_100


In [23]:
output_dir = "./text_generation_checkpoints"
model2 = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1) # [1, None]输入一个样本就可以生成文本
model2.load_weights(tf.train.latest_checkpoint(output_dir))
# 文本生成的流程
# start ch sequence A, 
# A -> model -> b  A放入模型后得到b
# A.append(b) -> B
# B(Ab) -> model -> c
# B.append(c) -> C
# C(Abc) -> model -> ...
print(model2.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            16640     
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (1, None, 1024)           1311744   
_________________________________________________________________
dense_1 (Dense)              (1, None, 65)             66625     
Total params: 1,395,009
Trainable params: 1,395,009
Non-trainable params: 0
_________________________________________________________________
None


In [24]:
# 文本生成流程
def generate_text(model,start_string,num_generate=1000):
    input_eval=[char2index[c] for c in start_string]
    print(input_eval)
    input_eval=tf.expand_dims(input_eval,0)
    print(input_eval)
    text_generated=[]
    # 初始状态 对model进行reset，连续调用的时候使用resets_states()
    model.reset_states()
    for _ in range(num_generate):
        predictions=model(input_eval)
        # [batch_size, input_eval_len, vocab_size]
        print(predictions.shape)
        predictions=tf.squeeze(predictions,0)
        # [input_eval_len, vocab_size]
        predicted_id=tf.random.categorical( # [input_eval_len, 1]
            predictions,num_samples=1)[-1,0].numpy()
        print(predicted_id)
        # 得到预测id后，放入text_generated
        text_generated.append(index2char[predicted_id])
        input_eval=tf.expand_dims([predicted_id],0) # [1, 1]-->model-->[1, 1, 65]
    return start_string+''.join(text_generated)

In [25]:
new_text=generate_text(model2,'All: ')
print(new_text)

[13, 50, 50, 10, 1]
tf.Tensor([[13 50 50 10  1]], shape=(1, 5), dtype=int32)
(1, 5, 65)
45
(1, 1, 65)
47
(1, 1, 65)
56
(1, 1, 65)
50
(1, 1, 65)
1
(1, 1, 65)
51
(1, 1, 65)
47
(1, 1, 65)
45
(1, 1, 65)
46
(1, 1, 65)
58
(1, 1, 65)
1
(1, 1, 65)
21
(1, 1, 65)
12
(1, 1, 65)
0
(1, 1, 65)
0
(1, 1, 65)
31
(1, 1, 65)
43
(1, 1, 65)
56
(1, 1, 65)
60
(1, 1, 65)
39
(1, 1, 65)
52
(1, 1, 65)
58
(1, 1, 65)
10
(1, 1, 65)
0
(1, 1, 65)
35
(1, 1, 65)
46
(1, 1, 65)
39
(1, 1, 65)
58
(1, 1, 65)
1
(1, 1, 65)
46
(1, 1, 65)
39
(1, 1, 65)
58
(1, 1, 65)
46
(1, 1, 65)
1
(1, 1, 65)
51
(1, 1, 65)
39
(1, 1, 65)
49
(1, 1, 65)
43
(1, 1, 65)
1
(1, 1, 65)
51
(1, 1, 65)
43
(1, 1, 65)
1
(1, 1, 65)
51
(1, 1, 65)
59
(1, 1, 65)
41
(1, 1, 65)
46
(1, 1, 65)
1
(1, 1, 65)
58
(1, 1, 65)
46
(1, 1, 65)
43
(1, 1, 65)
1
(1, 1, 65)
39
(1, 1, 65)
56
(1, 1, 65)
51
(1, 1, 65)
63
(1, 1, 65)
1
(1, 1, 65)
45
(1, 1, 65)
53
(1, 1, 65)
52
(1, 1, 65)
43
(1, 1, 65)
8
(1, 1, 65)
0
(1, 1, 65)
0
(1, 1, 65)
14
(1, 1, 65)
33
(1, 1, 65)
15
(1, 1, 65)
23
